# 训练自己的分类器

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf

from tensorflow.keras.preprocessing import image_dataset_from_directory

In [ ]:
model_name='3_ResNetv3Large_20120826_085751'
Model_AutoSaving_path='./AutoSaved_models./'+model_name

In [ ]:
PATH='E:\programming\data'

train_dir=os.path.join(PATH,'img')
#val_dir=os.path.join(PATH,'val')

In [ ]:
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

---

In [ ]:
train_dataset=image_dataset_from_directory(train_dir,
                                            shuffle=True,
                                            batch_size=BATCH_SIZE,
                                            image_size=IMG_SIZE)

In [ ]:
class_names=train_dataset.class_names

plt.figure(figsize=(20,20))
for images,labels in train_dataset.take(1):
    for i in range(30):
        ax=plt.subplot(6,5,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis('off')

In [ ]:
class_names

In [ ]:
AUTOTUNE=tf.data.AUTOTUNE

train_dataset=train_dataset.prefetch(buffer_size=AUTOTUNE)

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
for image,_ in train_dataset.take(1):
    plt.figure(figsize=(10,10))
    first_image=image[0]
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        augmented_image=data_augmentation(tf.expand_dims(first_image,0))
        plt.imshow(augmented_image[0]/255.)
        plt.axis('off')

In [ ]:
preprocess_input=tf.keras.applications.mobilenet_v3.preprocess_input

In [ ]:
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [ ]:
# Create the base model from the pre-trained model MobileNet V2
IMG_SHAPE=IMG_SIZE+(3,)
base_model=tf.keras.applications.MobileNetV3Large(input_shape=IMG_SHAPE,
                                                include_top=False,
                                                weights='imagenet')

In [ ]:
image_batch,label_batch=next(iter(train_dataset))
feature_batch=base_model(image_batch)
print(feature_batch.shape)

In [ ]:
base_model.trainable=False

In [ ]:
base_model.summary()

In [ ]:
global_average_layer=tf.keras.layers.GlobalAveragePooling2D()
feature_batch_average=global_average_layer(feature_batch)
print(feature_batch_average.shape)

In [ ]:
prediction_layer=tf.keras.layers.Dense(
    len(class_names),
    name='prediction_layer'
)
prediction_batch=prediction_layer(feature_batch_average)
print(prediction_batch.shape)

In [ ]:
inputs=tf.keras.Input(shape=(224,224,3))
x=data_augmentation(inputs)
x=preprocess_input(x)
x=base_model(x,training=False)
x=global_average_layer(x)
x=tf.keras.layers.Dropout(0.2)(x)
outputs=prediction_layer(x)

model=tf.keras.Model(inputs,outputs,name=model_name)

In [ ]:
base_learning_rate=0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
root_logdir=os.path.join(os.curdir,'explore_logs')

def get_run_logdir():
    import time
    run_id=time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir,run_id)

run_logdir=get_run_logdir()
tensorboard_cb=tf.keras.callbacks.TensorBoard(run_logdir)

In [ ]:
class Model_AutoSaving(tf.keras.callbacks.Callback):
    def on_epoch_end(self,epoch,logs):
        self.model.save(os.path.join(Model_AutoSaving_path,"epoch_%d.h5"%epoch))

In [ ]:
initial_epochs=10

history=model.fit(train_dataset,
                    epochs=initial_epochs,
                    callbacks=[Model_AutoSaving()]
)

In [ ]:
acc = history.history['accuracy']
#val_acc = history.history['val_accuracy']

loss = history.history['loss']
#val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
#plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
#plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

In [ ]:
# Fine-tune from this layer onwards
fine_tune_at = 200

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.RMSprop(learning_rate=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_dataset,
                        epochs=total_epochs,
                        initial_epoch=history.epoch[-1],
                        callbacks=[Model_AutoSaving()]
)

In [ ]:
acc += history_fine.history['accuracy']

loss += history_fine.history['loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

## save model

In [ ]:
model.save('./trained_models/ResNetv3Large_20210812_141921')

In [ ]:
print(class_names)

In [ ]:
#Retrieve a batch of images from the test set
image_batch, label_batch = train_dataset.as_numpy_iterator().next()
predictions = model.predict_on_batch(image_batch)

# Apply a sigmoid since our model returns logits
predictions = tf.nn.softmax(predictions)
'''predictions = tf.where(predictions < 0.5, 0, 1)'''
predictions=predictions.numpy()
sub=np.argmax(predictions,axis=1)
pos=np.max(predictions,axis=1)

print('Predictions:\n', sub)
print('Labels:\n', label_batch)

plt.figure(figsize=(15, 30))
for i in range(30):
  ax = plt.subplot(10, 3, i + 1)
  plt.imshow(image_batch[i].astype("uint8"))
  plt.title(str(class_names[sub[i]])+':%.2f%%'%(pos[i]*100))
  plt.axis("off")